# GloVE

Let's work on implementation of GloVE.

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

## 1. Load data

In [2]:
from nltk.corpus import brown

brown.categories()

#returns a list of sentences, where each sentence is a list of words from the "news" category of the Brown Corpus
corpus = brown.sents(categories="news")
corpus

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [3]:
#get word sequences and unique words
flatten = lambda l: [item for sublist in l for item in sublist]
vocabs = list(set(flatten(corpus)))
vocabs

['characteristic',
 'permit',
 'causes',
 'squeeze',
 'Congo',
 '1825',
 'Tulsa',
 'voice',
 'paths',
 'captain',
 'Oslo',
 'Connall',
 'isolated',
 'Plainfield',
 "McClellan's",
 '$4,177.37',
 'statutory',
 'Implementation',
 'too',
 'private-school',
 '355',
 'low-wage',
 'grass',
 'avoids',
 'inject',
 'blueprints',
 'stakes',
 'labeled',
 'About',
 'manipulate',
 'schools',
 'Tires',
 'influences',
 'picture',
 'prepare',
 'strategy',
 'produce',
 'gasoline',
 "Mantle's",
 'Birmingham',
 'inducted',
 '1%',
 'Orioles',
 'Ferris',
 'Marsicano',
 'toneless',
 'sirens',
 'Ranch',
 '$67,000',
 'threaten',
 'Wide',
 '$12,500',
 'busy-work',
 'Senese',
 'Avenue',
 'excitement',
 'ready',
 'barn',
 'records',
 'shown',
 'super-secret',
 'Hatters',
 'ugly',
 'Afterwards',
 'socket',
 'kidnaped',
 'Baker',
 'load',
 'behalf',
 'propelling',
 '675',
 'Allen',
 'parolees',
 '4:05',
 'anymore',
 'CD',
 'Fireside',
 "Hansen's",
 '1957',
 'lies',
 'fictitious',
 'poultry',
 'caucuses',
 'theater'

In [4]:
#numericalization
word2index = {w: i for i, w in enumerate(vocabs)}
print(word2index)

{'characteristic': 0, 'permit': 1, 'causes': 2, 'squeeze': 3, 'Congo': 4, '1825': 5, 'Tulsa': 6, 'voice': 7, 'paths': 8, 'captain': 9, 'Oslo': 10, 'Connall': 11, 'isolated': 12, 'Plainfield': 13, "McClellan's": 14, '$4,177.37': 15, 'statutory': 16, 'Implementation': 17, 'too': 18, 'private-school': 19, '355': 20, 'low-wage': 21, 'grass': 22, 'avoids': 23, 'inject': 24, 'blueprints': 25, 'stakes': 26, 'labeled': 27, 'About': 28, 'manipulate': 29, 'schools': 30, 'Tires': 31, 'influences': 32, 'picture': 33, 'prepare': 34, 'strategy': 35, 'produce': 36, 'gasoline': 37, "Mantle's": 38, 'Birmingham': 39, 'inducted': 40, '1%': 41, 'Orioles': 42, 'Ferris': 43, 'Marsicano': 44, 'toneless': 45, 'sirens': 46, 'Ranch': 47, '$67,000': 48, 'threaten': 49, 'Wide': 50, '$12,500': 51, 'busy-work': 52, 'Senese': 53, 'Avenue': 54, 'excitement': 55, 'ready': 56, 'barn': 57, 'records': 58, 'shown': 59, 'super-secret': 60, 'Hatters': 61, 'ugly': 62, 'Afterwards': 63, 'socket': 64, 'kidnaped': 65, 'Baker': 

In [5]:
#vocab size
voc_size = len(vocabs)
print(voc_size)

14394


In [6]:
#append UNK
vocabs.append('<UNK>')

In [7]:
vocabs

['characteristic',
 'permit',
 'causes',
 'squeeze',
 'Congo',
 '1825',
 'Tulsa',
 'voice',
 'paths',
 'captain',
 'Oslo',
 'Connall',
 'isolated',
 'Plainfield',
 "McClellan's",
 '$4,177.37',
 'statutory',
 'Implementation',
 'too',
 'private-school',
 '355',
 'low-wage',
 'grass',
 'avoids',
 'inject',
 'blueprints',
 'stakes',
 'labeled',
 'About',
 'manipulate',
 'schools',
 'Tires',
 'influences',
 'picture',
 'prepare',
 'strategy',
 'produce',
 'gasoline',
 "Mantle's",
 'Birmingham',
 'inducted',
 '1%',
 'Orioles',
 'Ferris',
 'Marsicano',
 'toneless',
 'sirens',
 'Ranch',
 '$67,000',
 'threaten',
 'Wide',
 '$12,500',
 'busy-work',
 'Senese',
 'Avenue',
 'excitement',
 'ready',
 'barn',
 'records',
 'shown',
 'super-secret',
 'Hatters',
 'ugly',
 'Afterwards',
 'socket',
 'kidnaped',
 'Baker',
 'load',
 'behalf',
 'propelling',
 '675',
 'Allen',
 'parolees',
 '4:05',
 'anymore',
 'CD',
 'Fireside',
 "Hansen's",
 '1957',
 'lies',
 'fictitious',
 'poultry',
 'caucuses',
 'theater'

In [8]:
word2index['<UNK>'] = 0

In [9]:
#just in case we need to use
index2word = {v:k for k, v in word2index.items()} 

## 2. Build Co-occurence Matrix X

Here, we need to count the co-occurence of two words given some window size.  We gonna use window size of 1.

In [10]:
from collections import Counter

X_i = Counter(flatten(corpus))
X_i

Counter({'the': 5580,
         ',': 5188,
         '.': 4030,
         'of': 2849,
         'and': 2146,
         'to': 2116,
         'a': 1993,
         'in': 1893,
         'for': 943,
         'The': 806,
         'that': 802,
         '``': 732,
         'is': 732,
         'was': 717,
         "''": 702,
         'on': 657,
         'at': 598,
         'with': 545,
         'be': 526,
         'by': 497,
         'as': 481,
         'he': 451,
         'said': 402,
         'his': 399,
         'will': 389,
         'it': 363,
         'from': 344,
         'are': 328,
         ';': 314,
         'an': 300,
         'has': 300,
         '--': 300,
         'had': 279,
         'who': 268,
         'have': 265,
         'not': 254,
         'Mrs.': 253,
         'were': 252,
         'this': 250,
         'which': 244,
         'would': 244,
         'their': 219,
         'been': 212,
         'they': 205,
         'He': 191,
         'one': 184,
         'I': 179,
         'but'

In [11]:
window_size = 2

skip_grams = []

#loop each corpus
for doc in corpus:
    #look from the 2nd word until second last word
    for i in range(window_size, len(doc)-window_size):
        #center word
        center = doc[i]
        #outside words = 4 words
        outside = []
        for j in range(window_size):
            outside.append(doc[i+(j+1)])
            outside.append(doc[i-(j+1)])

        #for each of these two outside words, we gonna append to a list
        for each_out in outside:
            skip_grams.append((center, each_out))
            #center, outside1;   center, outside2

skip_grams

[('County', 'Grand'),
 ('County', 'Fulton'),
 ('County', 'Jury'),
 ('County', 'The'),
 ('Grand', 'Jury'),
 ('Grand', 'County'),
 ('Grand', 'said'),
 ('Grand', 'Fulton'),
 ('Jury', 'said'),
 ('Jury', 'Grand'),
 ('Jury', 'Friday'),
 ('Jury', 'County'),
 ('said', 'Friday'),
 ('said', 'Jury'),
 ('said', 'an'),
 ('said', 'Grand'),
 ('Friday', 'an'),
 ('Friday', 'said'),
 ('Friday', 'investigation'),
 ('Friday', 'Jury'),
 ('an', 'investigation'),
 ('an', 'Friday'),
 ('an', 'of'),
 ('an', 'said'),
 ('investigation', 'of'),
 ('investigation', 'an'),
 ('investigation', "Atlanta's"),
 ('investigation', 'Friday'),
 ('of', "Atlanta's"),
 ('of', 'investigation'),
 ('of', 'recent'),
 ('of', 'an'),
 ("Atlanta's", 'recent'),
 ("Atlanta's", 'of'),
 ("Atlanta's", 'primary'),
 ("Atlanta's", 'investigation'),
 ('recent', 'primary'),
 ('recent', "Atlanta's"),
 ('recent', 'election'),
 ('recent', 'of'),
 ('primary', 'election'),
 ('primary', 'recent'),
 ('primary', 'produced'),
 ('primary', "Atlanta's"),
 (

In [12]:
X_ik_skipgrams = Counter(skip_grams)
X_ik_skipgrams

Counter({('the', 'of'): 1460,
         ('of', 'the'): 1443,
         ('the', ','): 870,
         (',', 'the'): 855,
         (',', ','): 809,
         ('the', 'in'): 663,
         ('in', 'the'): 659,
         (',', 'and'): 624,
         ('and', ','): 624,
         ('the', 'to'): 594,
         ('to', 'the'): 590,
         ('of', ','): 380,
         (',', 'of'): 377,
         ('and', 'the'): 357,
         ('the', '.'): 354,
         ('the', 'and'): 353,
         ('of', 'a'): 351,
         ('a', 'of'): 346,
         (',', 'a'): 324,
         ('a', ','): 319,
         ('the', 'for'): 280,
         (',', 'in'): 280,
         ('in', ','): 280,
         ('for', 'the'): 279,
         ('the', 'on'): 277,
         ('on', 'the'): 274,
         (',', 'said'): 250,
         ('the', 'that'): 224,
         ('that', 'the'): 222,
         ('to', 'a'): 221,
         ('a', 'to'): 219,
         ('the', 'at'): 210,
         (',', "''"): 209,
         ('at', 'the'): 208,
         ("''", ','): 207,
         

### Weighting function

GloVe includes a weighting function to scale down too frequent words.

<img src = "../figures/glove_weighting_func.png" width=400>

In [13]:
def weighting(w_i, w_j, X_ik):
    
    #check whether the co-occurences between w_i and w_j is available
    try:
        x_ij = X_ik[(w_i, w_j)]
        #if not exist, then set to 1 "laplace smoothing"
    except:
        x_ij = 1
        
    #set xmax
    x_max = 100
    #set alpha
    alpha = 0.75
    
    #if co-ocurrence does not exceeed xmax, then just multiply with some alpha
    if x_ij < x_max:
        result = (x_ij / x_max)**alpha
    #otherwise, set to 1
    else:
        result = 1
    
    return result

In [14]:
from itertools import combinations_with_replacement

X_ik = {} #keeping the co-occurences
weighting_dic = {} #already scale the co-occurences using the weighting function

for bigram in combinations_with_replacement(vocabs, 2):
    if X_ik_skipgrams.get(bigram):  #if the pair exists in our corpus
        co = X_ik_skipgrams[bigram]
        X_ik[bigram] = co + 1 #for stability
        X_ik[(bigram[1], bigram[0])] = co + 1 #basically apple, banana = banana, apple
    else:
        pass
    
    weighting_dic[bigram] = weighting(bigram[0], bigram[1], X_ik)
    weighting_dic[(bigram[1], bigram[0])] = weighting(bigram[1], bigram[0], X_ik)

## 3. Prepare train data

In [15]:
import math

def random_batch(batch_size, word_sequence, skip_grams, X_ik, weighting_dic):
    
    random_inputs, random_labels, random_coocs, random_weightings = [], [], [], []
    
    #convert our skipgrams to id
    skip_grams_id = [(word2index[skip_gram[0]], word2index[skip_gram[1]]) for skip_gram in skip_grams]
    
    #randomly choose indexes based on batch size
    random_index = np.random.choice(range(len(skip_grams_id)), batch_size, replace=False)
    
    #get the random input and labels
    for index in random_index:
        random_inputs.append([skip_grams_id[index][0]])
        random_labels.append([skip_grams_id[index][1]])
        #coocs
        pair = skip_grams[index] #e.g., ('banana', 'fruit')
        try:
            cooc = X_ik[pair]
        except:
            cooc = 1
        random_coocs.append([math.log(cooc)])
    
        #weightings
        weighting = weighting_dic[pair]
        random_weightings.append([weighting])
        
    return np.array(random_inputs), np.array(random_labels), np.array(random_coocs), np.array(random_weightings)

### Testing the method

In [16]:
batch_size = 2
x, y, cooc, weighting = random_batch(batch_size, corpus, skip_grams, X_ik, weighting_dic)

In [17]:
x

array([[3443],
       [ 588]])

In [18]:
y

array([[5553],
       [4211]])

In [19]:
cooc

array([[1.38629436],
       [1.09861229]])

In [20]:
weighting

array([[0.08944272],
       [0.07208434]])

## 4. Model

<img src ="../figures/glove.png" width=400>

In [21]:
class Glove(nn.Module):
    
    def __init__(self, voc_size, emb_size, word2index):
        super(Glove, self).__init__()
        self.center_embedding  = nn.Embedding(voc_size, emb_size)
        self.outside_embedding = nn.Embedding(voc_size, emb_size)
        self.word2index        = word2index
        
        self.center_bias       = nn.Embedding(voc_size, 1) 
        self.outside_bias      = nn.Embedding(voc_size, 1)
    
    def forward(self, center, outside, coocs, weighting):
        center_embeds  = self.center_embedding(center) #(batch_size, 1, emb_size)
        outside_embeds = self.outside_embedding(outside) #(batch_size, 1, emb_size)
        
        center_bias    = self.center_bias(center).squeeze(1)
        target_bias    = self.outside_bias(outside).squeeze(1)
        
        inner_product  = outside_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #(batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) = (batch_size, 1)
        
        loss = weighting * torch.pow(inner_product + center_bias + target_bias - coocs, 2)
        
        return torch.sum(loss)
    
    def get_embed(self, word):
        word2index = self.word2index
        
        try:
            index = word2index[word]
        except:
            index = word2index['<UNK>']
            
        word = torch.LongTensor([index])
        
        embed_c = self.center_embedding(word)
        embed_o = self.outside_embedding(word)
        embed   = (embed_c + embed_o) / 2
        
        return embed[0][0].item(), embed[0][1].item()

In [22]:
#test our system
voc_size = len(vocabs)
emb_size = 2
model = Glove(voc_size, emb_size, word2index)

In [23]:
x_tensor = torch.LongTensor(x)
y_tensor = torch.LongTensor(y)
cooc_tensor = torch.FloatTensor(cooc)
weighting_tensor = torch.FloatTensor(weighting)

In [24]:
loss = model(x_tensor, y_tensor, cooc_tensor, weighting_tensor)

In [25]:
loss

tensor(0.5641, grad_fn=<SumBackward0>)

## 5. Training

In [26]:
batch_size     = 10 # mini-batch size
embedding_size = 2 #so we can later plot
model          = Glove(voc_size, embedding_size, word2index)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [28]:
import time

# Training
num_epochs = 1000

start = time.time()

for epoch in range(num_epochs):
    
    input_batch, target_batch, cooc_batch, weighting_batch = random_batch(batch_size, corpus, skip_grams, X_ik, weighting_dic)
    input_batch  = torch.LongTensor(input_batch)         #[batch_size, 1]
    target_batch = torch.LongTensor(target_batch)        #[batch_size, 1]
    cooc_batch   = torch.FloatTensor(cooc_batch)         #[batch_size, 1]
    weighting_batch = torch.FloatTensor(weighting_batch) #[batch_size, 1]
    
    optimizer.zero_grad()
    loss = model(input_batch, target_batch, cooc_batch, weighting_batch)
    
    loss.backward()
    optimizer.step()
    
    end = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start, end)

    if (epoch + 1) % 100 == 0:
        print(f"Epoch: {epoch + 1} | cost: {loss:.6f} | time: {epoch_mins}m {epoch_secs}s")


Epoch: 100 | cost: 18.867945 | time: 0m 4s
Epoch: 200 | cost: 8.801160 | time: 0m 9s
Epoch: 300 | cost: 49.558479 | time: 0m 14s
Epoch: 400 | cost: 97.290710 | time: 0m 19s
Epoch: 500 | cost: 35.039146 | time: 0m 23s
Epoch: 600 | cost: 49.048317 | time: 0m 28s
Epoch: 700 | cost: 21.116968 | time: 0m 33s
Epoch: 800 | cost: 61.768444 | time: 0m 38s
Epoch: 900 | cost: 90.194122 | time: 0m 43s
Epoch: 1000 | cost: 72.682037 | time: 0m 48s


In [29]:
def open_file(path_to_file):
    # Open the file in read mode
    try:
        with open(path_to_file, 'r') as file:
            content = file.readlines()
    except FileNotFoundError:
        print(f"The file {path_to_file} does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return content

In [30]:
file_path_semantic = "../Test/word-test.v1-capital.txt"
file_path_syntatic = "../Test/word-test.v1-past-tense.txt"

content_semantic = open_file(file_path_semantic)
content_syntactic = open_file(file_path_syntatic)

semantic = []
syntactic = []

for sent in content_semantic:
    semantic.append(sent.strip())

for sent in content_syntactic:
    syntactic.append(sent.strip())

In [31]:
vector_space = []

for word in vocabs:
    vector_space.append(model.get_embed(word))

vector_space = np.array(vector_space)

In [32]:
#scipy version
from scipy import spatial

def cos_sim(a, b):
    cos_sim = 1 - spatial.distance.cosine(a, b)  #distance = 1 - similarlity, because scipy only gives distance
    return cos_sim

def cos_sim_scores(vector_space, target_vector):
    scores = []
    for each_vect in vector_space:
        each_vect = tuple(each_vect)
        target_vector=tuple(target_vector)
        scores.append(cos_sim(target_vector, each_vect))

    return np.array(scores)

In [33]:
def similarity(model, test_data):
    words = test_data.split(" ")

    embed0 = np.array(model.get_embed(words[0]))
    embed1 = np.array(model.get_embed(words[1]))
    embed2 = np.array(model.get_embed(words[2]))

    similar_vector = embed1 - embed0 + embed2

    similarity_scores = cos_sim_scores(vector_space, similar_vector)
    max_score_idx = np.argmax(similarity_scores)
    similar_word = index2word[max_score_idx]

    result = False
    if similar_word == words[3]:
        result = True

    return result

## Semantic accuracy

In [34]:
sem_total = len(semantic)
sem_correct = 0
for sent in semantic:
    if similarity(model, sent):
        sem_correct += 1

In [35]:
sem_accuracy = sem_correct / sem_total
print(f"Semantic accuracy: {sem_accuracy:2.2f}")

Semantic accuracy: 0.00


## Syntactixc

In [36]:
syn_total = len(syntactic)
syn_correct = 0
for sent in syntactic:
    if similarity(model, sent):
        syn_correct += 1

In [37]:
syn_accuracy = syn_correct / syn_total
print(f"Syntactic accuracy: {syn_accuracy:2.2f}")

Syntactic accuracy: 0.00


## Similarity Accuracy

In [38]:
file_path = "../Test/wordsim_similarity_goldstandard.txt"

content = open_file(file_path)

sim_data = []

for sent in content:
    sim_data.append(sent.strip())

In [39]:
def compute_similarity(model, test_data):
    words = test_data.split("\t")

    embed0 = np.array(model.get_embed(words[0].strip()))
    embed1 = np.array(model.get_embed(words[1].strip()))

    similarity_model = embed1 @ embed0.T
    similarity_provided = float(words[2].strip())

    return similarity_provided, similarity_model

In [40]:
ds_scores = []
model_scores = []
for sent in sim_data:
    ds_score, model_score = compute_similarity(model, sent)

    ds_scores.append(ds_score)
    model_scores.append(model_score)

In [41]:
from scipy.stats import spearmanr

corr = spearmanr(ds_scores, model_scores)[0]

print(f"Correlation between the dataset metrics and model scores is {corr:2.2f}.")

Correlation between the dataset metrics and model scores is 0.21.
